In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [18]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim

### Expanded Function

In [3]:
Nx = 2
Ne = 8
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';
B = MultiBasis(CstProHermite(3; scaled =true), Nx)

idx = [0 0; 0 1; 1 0; 2 1; 1 2]
truncidx = idx[1:2:end,:]
Nψ = 5

coeff =  [0.6285037650645056;
 -0.4744029092496623;
  1.1405280011620331;
 -0.7217760771894809;
  0.11855056306742319]
f = ExpandedFunction(B, idx, coeff)

fp = ParametricFunction(f)
R = IntegratedFunction(fp)

H = HermiteMapk(R)


S = Storage(H.I.f, ens.S);

In [4]:
X = deepcopy(ens.S)
xk = X[Nx,:];

In [29]:
negative_log_likelihood!(S, H, ens.S)(0.0, zeros(Nψ), randn(Nψ))

1.27520711657622

In [30]:
coeff = randn(Nψ)

5-element Array{Float64,1}:
 -0.043509926772964856
 -1.0724826940522136
  0.3342120227752588
 -0.3871935499945769
  2.1872332588211343

In [31]:
res = Optim.optimize(negative_log_likelihood!(S, H, ens.S), coeff, Optim.BFGS())
@time Optim.minimizer(res)

ArgumentError: ArgumentError: Value and slope at step length = 0 must be finite.

In [32]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, H, ens.S)), coeff, Optim.BFGS())
@time Optim.minimizer(res)

ArgumentError: ArgumentError: Value and slope at step length = 0 must be finite.